In [1]:
import os
import shutil
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator

dir_list = ['boron-B',
 'calcium-Ca',
#  'healthy',
 'iron-Fe',
 'magnesium-Mg',
 'manganese-Mn',
 'more-deficiencies',
 'nitrogen-N',
 'phosphorus-P',
 'potasium-K']

for classes in dir_list:
    
    kaggle_input_dir = '/kaggle/input/coffee-leaf-disease-akash/CoLeaf DATASET/' + classes

    temp_dir = '/kaggle/working/temp/' + classes

    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)

    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest'
    )

    generated_images = datagen.flow_from_directory(
        kaggle_input_dir,
        save_to_dir=temp_dir,
        batch_size = 1,
        save_prefix=classes,
        save_format='jpg'
    )

    num_images = 700
    for i in range(num_images):
        generated_image = next(generated_images)


    shutil.make_archive('/kaggle/temp/' + classes, 'zip', temp_dir)


2024-05-21 07:22:10.115662: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 07:22:10.115775: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 07:22:10.208633: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 101 images belonging to 1 classes.
Found 162 images belonging to 1 classes.
Found 65 images belonging to 1 classes.
Found 79 images belonging to 1 classes.
Found 83 images belonging to 1 classes.
Found 104 images belonging to 1 classes.
Found 64 images belonging to 1 classes.
Found 246 images belonging to 1 classes.
Found 96 images belonging to 1 classes.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import cv2
import os

    # Define dataset paths
dataset_path = '/kaggle/working/temp'
categories = os.listdir(dataset_path)
num_classes = len(categories)

    # Load images and labels
def load_data():
    images = []
    labels = []
    for label, category in enumerate(categories):
        category_path = os.path.join(dataset_path, category)
        for root, _, files in os.walk(category_path):
            for img_name in files:
                img_path = os.path.join(root, img_name)
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.resize(img, (224, 224))  # Resize images to 224x224
                    images.append(img)
                    labels.append(label)
                else:
                    print(f"Failed to load image: {img_path}")
    return np.array(images), np.array(labels)

images, labels = load_data()

# Ensure that we have loaded images
if len(images) == 0:
    raise Exception("No images loaded. Check dataset path and image files.")
    
images = images / 255.0


# Convert labels to one-hot encoding
labels = to_categorical(labels, num_classes)



In [3]:
labels.shape

(6300, 9)

In [4]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam

import math
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from sklearn.model_selection import train_test_split

datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)


def create_densenet121_model(input_shape, num_classes):
    base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)  # Regularization
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    
    for layer in base_model.layers[:-30]:
        layer.trainable = False

    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

densenet121_model = create_densenet121_model(input_shape=(224, 224, 3), num_classes=num_classes)


def step_decay(epoch):
    initial_lr = 0.001
    drop = 0.5
    epochs_drop = 10.0
    lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lr

lr_scheduler = LearningRateScheduler(step_decay)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=3, min_lr=0.00001)


from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Define callbacks
callbacks = [
    ModelCheckpoint('densenet121_best_model.keras', save_best_only=True, monitor='val_accuracy', mode='max'),
    EarlyStopping(monitor='val_accuracy', mode='max', patience=5, verbose=1),
    lr_scheduler,
    reduce_lr
]



# Train the model
history = densenet121_model.fit(
    datagen.flow(X_train, y_train, batch_size=42),
    validation_data=(X_val, y_val),
    epochs=30,
    callbacks=callbacks
)

# Evaluate the model
scores = densenet121_model.evaluate(X_val, y_val, verbose=0)
print(f'Validation Loss: {scores[0]}, Validation Accuracy: {scores[1]}')

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/30


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  1/120 ━━━━━━━━━━━━━━━━━━━━ 1:42:39 52s/step - accuracy: 0.0238 - loss: 2.8194

I0000 00:00:1716276997.793371     191 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1716276997.869455     191 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 491ms/step - accuracy: 0.3990 - loss: 1.8101

W0000 00:00:1716277081.840092     188 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1716277099.546037     191 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


120/120 ━━━━━━━━━━━━━━━━━━━━ 154s 862ms/step - accuracy: 0.4000 - loss: 1.8067 - val_accuracy: 0.6746 - val_loss: 0.9104 - learning_rate: 0.0010
Epoch 2/30
120/120 ━━━━━━━━━━━━━━━━━━━━ 67s 529ms/step - accuracy: 0.6337 - loss: 0.9866 - val_accuracy: 0.7548 - val_loss: 0.6901 - learning_rate: 0.0010
Epoch 3/30
120/120 ━━━━━━━━━━━━━━━━━━━━ 66s 518ms/step - accuracy: 0.7012 - loss: 0.8276 - val_accuracy: 0.7468 - val_loss: 0.6775 - learning_rate: 0.0010
Epoch 4/30
120/120 ━━━━━━━━━━━━━━━━━━━━ 67s 523ms/step - accuracy: 0.7545 - loss: 0.6801 - val_accuracy: 0.7849 - val_loss: 0.6025 - learning_rate: 0.0010
Epoch 5/30
120/120 ━━━━━━━━━━━━━━━━━━━━ 66s 522ms/step - accuracy: 0.7602 - loss: 0.6650 - val_accuracy: 0.8175 - val_loss: 0.5026 - learning_rate: 0.0010
Epoch 6/30
120/120 ━━━━━━━━━━━━━━━━━━━━ 66s 516ms/step - accuracy: 0.8006 - loss: 0.5466 - val_accuracy: 0.8111 - val_loss: 0.5217 - learning_rate: 0.0010
Epoch 7/30
120/120 ━━━━━━━━━━━━━━━━━━━━ 68s 539ms/step - accuracy: 0.8286 - loss

KeyboardInterrupt: 

In [5]:
loss, accuracy = densenet121_model.evaluate(X_val, y_val)
accuracy

40/40 ━━━━━━━━━━━━━━━━━━━━ 3s 78ms/step - accuracy: 0.9473 - loss: 0.1156


0.946825385093689